In [14]:
# Install libraries
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import json
import time, random
import pymysql.cursors
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
# Open json with keywords
with open("config_keys.json") as f:
    config_keys = json.load(f)

In [16]:
# Set the path to your ChromeDriver executable
chrome_driver_path = 'C:\\Users\\Eugenia.Primo\\Desktop\\chromedriver.exe'

# Set the URL of the Indeed page you want to scrape
countries = ["uk", "it", "es", "de", "fr"]
titles = []
company_names = []
locations = [] 
job_salary = []
url_country = []

for country in countries: 
    url = f'https://{country}.indeed.com/'

    # Set the job title and location you want to search for
    job_title = config_keys['search_queries']["keywords"]
    location = config_keys['search_queries']["location"]

    # Initialize Chrome WebDriver
    cService = webdriver.ChromeService(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service = cService)

    # Open Indeed website
    driver.get(url)
    
    for key_job in job_title: 
        for key_loc in location: 
            try:   
                 # Find the search boxes for job title and location
                what_input = driver.find_element(By.NAME, 'q')
                where_input = driver.find_element(By.NAME, 'l')
                # Type in job title and location and press Enter
                what_input.click()
                what_input.send_keys(Keys.CONTROL + 'a')  # Select all text in the input box
                what_input.send_keys(Keys.DELETE) 
                what_input.send_keys(key_job)

                # Clear any prewritten text in the location input box
                where_input.click()
                where_input.send_keys(Keys.CONTROL + 'a')  # Select all text in the input box
                where_input.send_keys(Keys.DELETE)         # Delete the selected text
                where_input.send_keys(key_loc)
                # Type in the desired location and press Enter
                where_input.send_keys(Keys.RETURN)
            except: 
                pop_up = driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/button").click()
                print(pop_up)
            
            # Set the maximum number of pages to iterate over
            max_pages = 5  
            page_counter = 0 

            while True:
                # Wait for the page to load
                time.sleep(3)

                # Find all job listings
                job_listings = driver.find_element(By.XPATH, '//*[@id="mosaic-provider-jobcards"]')
                job_listings = job_listings.get_attribute('innerHTML')
                soup = BeautifulSoup(job_listings, 'html.parser')

                job_cards = soup.find_all('div', class_='job_seen_beacon')

                for job in job_cards: 
                    try: 
                        title = job.find('h2', class_='jobTitle').find('span').text
                    except: 
                        title = "Not Specified"

                    try:
                        name = job.find('div', class_='company_location').find('span', class_='css-92r8pb eu4oa1w0').text
                    except: 
                        name = "Not Specified" 

                    try:
                        job_location = job.find('div', class_='company_location').find('div', class_='css-1p0sjhy eu4oa1w0').find('span').text
                    except: 
                        job_location = "Not Specified"  

                    try:
                        salary = job.find('div', class_='css-1cvo3fd eu4oa1w0').text.strip()
                    except: 
                        salary = "Not Specified"  

                    print(key_job, key_loc, title, name, job_location, salary)

                    titles.append(title)
                    company_names.append(name)
                    locations.append(job_location)
                    job_salary.append(salary)
                    url_country.append(country)

                try:
                    # Find the next button:
                    next_page_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//a[@data-testid="pagination-page-next"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", next_page_link) # Scroll to the next page link
                    driver.implicitly_wait(1) # Wait to ensure the page has finished scrolling
                    next_page_link.click()
                    page_counter += 1
                    
                    if page_counter >= max_pages:
                        print("Maximum pages reached. Exiting loop.")
                        break

                except: 
                    print("no more pages")
                    break
                
        # Save data extraction in database
        df_hist = pd.read_csv("database/indeed_jobs.csv") # historical data

        df_new = pd.DataFrame({"job_title": titles, "company_name": company_names, "location": locations, "salary": job_salary, "url_country": url_country})
        df_new = df_new[~df_new["company_name"].isin(df_hist["company_name"])]
        df_new["extraction_date"] = datetime.now()

        df = pd.concat([df_hist, df_new], axis = 0) # append new jobs to historical database
        df.to_csv("database/indeed_jobs.csv", index=False)

        company_list = df.drop_duplicates(subset = "company_name")
        company_list.to_csv("database/indeed_companies.csv", index=False)
        print("save in database")
        
# Close the browser window
driver.quit()

software developer remote Senior Software Developer First Databank Remote Not Specified
software developer remote Front End Developer Geronimo Web Remote Not Specified
software developer remote Front End Developer (100% Remote) Bitfinex Remote Not Specified
software developer remote Front-end Software Developer Harris Computer Systems Remote Not Specified
software developer remote Software Developer Good Growth Limited Hybrid remote Not Specified
software developer remote Python Developer FFW Remote Not Specified
software developer remote Remote Front End React Developer SchemeServe Remote Not Specified
software developer remote Python Developer Opus Recruitment Solutions Remote Not Specified
software developer remote Software Engineer Admiral Remote Not Specified
software developer remote Software Engineer Intern Smarsh Hybrid remote Not Specified
software developer remote Frontend Developer - Cairo Tangent Marketing Services Remote Not Specified
software developer remote Front End En

In [10]:

companies

,job_title,company_name,location,salary
0,Front End Developer,Geronimo Web,Remote,Not Specified
1,Software Engineer Intern,Smarsh,Hybrid remote,Not Specified
3,Software Developer,ao.com,Remote,Not Specified
4,"Software Engineer II, Front End",Dataminr,Remote,Not Specified
5,Remote Front End React Developer,SchemeServe,Remote,Not Specified
...,...,...,...,...
294,Data Engineer (m/f/d) at internationally growi...,Edurino,Homeoffice,Vollzeit
318,JavaScript Developer,smart IoT group,Homeoffice,Not Specified
319,Backend Developer (m/f/x),Torg GmbH,Homeoffice,Freie Mitarbeit
328,Senior Software Engineer - AI & Machine Learni...,Dataiku,Télétravail,Not Specified
